In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
%pip install torchinfo torch-summary matplotlib scipy opencv-python

In [ ]:
from pathlib import Path
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF
import cv2
import numpy as np


# TODO: call load_dataset instead of inline here

dataset_name = "cxr8"

base_data_path = Path("D:\\") / "data"

imagenet_data_path = (
    base_data_path
    / "imagenet-object-localization-challenge"
    / "ILSVRC"
    / "Data"
    / "CLS-LOC"
)

cxr8_data_path = base_data_path / "cxr8" / "filtered"

CLAHE_TILE = 8
clahe = cv2.createCLAHE(clipLimit=4, tileGridSize=(CLAHE_TILE, CLAHE_TILE))


# def applyClahe(tensor):
# print(type(tensor), tensor.size)
#   return clahe.apply(np.array(tensor))


input_size = (1, 448, 448)

if dataset_name == "cxr8":
    train_dataset = datasets.ImageFolder(
        cxr8_data_path,
        transform=transforms.Compose(
            [
                transforms.Resize((input_size[1], input_size[2])),
                transforms.Grayscale(),
                transforms.Lambda(np.array),
                transforms.Lambda(clahe.apply),
                transforms.ToTensor(),
            ]
        ),
    )
elif dataset_name == "cifar10":
    train_dataset = datasets.CIFAR10(
        root="./data",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize((224, 224)), transforms.ToTensor()]
        ),
    )
elif dataset_name == "imagenet":
    train_dataset = datasets.ImageFolder(
        imagenet_data_path,
        transform=transforms.Compose(
            [transforms.Resize((224, 224)), transforms.ToTensor()]
        ),
    )

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


In [ ]:
import torch
from torchinfo import summary

from vae import VAE, vae_loss

KERNEL_SIZE = 11
DIRECTIONS = 7
LATENT_DIM = 32  # 64
show_summary = True

# model = VAE((1 if dataset_name == "cxr8" else 3, 224, 224), latent_dim).to(device)
model = VAE(input_size, init_kernel_size=KERNEL_SIZE, latent_dim=LATENT_DIM)
if show_summary:
    print(
        summary(
            model,
            input_size=(37, input_size[0], input_size[1], input_size[2]),
            depth=10,
            col_names=[
                "input_size",
                "kernel_size",
                "mult_adds",
                "num_params",
                "output_size",
                "trainable",
            ],
        )
    )

device = torch.device("cuda" if torch.cuda.is_available else "cpu")
print(device)

model = model.to(device)
print(set([p.device for p in model.parameters()]))


In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from IPython.display import clear_output, display
import torch.nn.functional as F

num_epochs = 5

import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=1e-4)

fig, ax = plt.subplots(2, 5, figsize=(20,8))

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    train_count = 0
    for batch_idx, (data, _) in enumerate(train_loader):

        data = data.to(device)
        optimizer.zero_grad()

        recon_batch, mu, log_var = model(data)
        recon_loss, kldiv_loss, loss = vae_loss(
            recon_batch, data, mu, log_var, recon_loss_metric="l1_loss",  beta=0.25
        )

        if train_count % 10 == 0:
            orig_data = data.clone().cpu().detach().numpy()
            recon_data = recon_batch.cpu().detach().numpy()

            # print(v.shape)
            for n in range(5):
                ax[0][n].imshow(torch.movedim(torch.tensor(orig_data[n]), 0, -1), cmap='bone')
                ax[1][n].imshow(torch.movedim(torch.tensor(recon_data[n]), 0, -1), cmap='bone')

            clear_output(wait=True)

            display(plt.gcf())

        loss.backward()
        train_loss += loss.item()
        train_count += 1.0
        optimizer.step()

        print(
            f"Epoch [{epoch+1}/{num_epochs}], Batch: {batch_idx}, Loss: {train_loss / train_count:.6f} ({recon_loss:.6f}/{kldiv_loss:.6f})"
        )
        

In [ ]:
from datetime import datetime

when = datetime.now().strftime("%Y%m%d%H%M%S")
weight_path = (
    Path("weights")
    / f"{when}_clahe{CLAHE_TILE}_kernel{KERNEL_SIZE}_latent{LATENT_DIM}_orisq.zip"
)

torch.save(model.state_dict(), weight_path)
